In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
class TRG:
    def __init__(self, temp: float, chi: int):
        self.temp = temp
        self.chi = chi
        self.step = 0
        self.T, log_factor, n_spin = self.initial_TN(self.temp)
        self.log_factors = [log_factor]
        self.n_spins = [n_spin]
        self.dir = f"{int(100*temp)}//{chi}"
        os.makedirs(self.dir, exist_ok=True)
        self.output()

    def initial_TN(self, temp: float):
        beta = 1.0 / temp
        M = np.array([[np.sqrt(np.cosh(beta)), np.sqrt(np.sinh(beta))],
                      [np.sqrt(np.cosh(beta)), -np.sqrt(np.sinh(beta))]])

        T_np = np.einsum('si,sj,sk,sl->ijkl', M, M, M, M)
        T = T_np

        trT = np.einsum('ijij->', T)
        T /= trT
        log_factor = np.log(trT)
        n_spin = 1

        return T, log_factor, n_spin

    def update(self):
        d = self.T.shape[0]

        # SVD 1
        T_perm = self.T.transpose(0,1,2,3)
        T_mat = T_perm.reshape(d*d, d*d)
        U, S, Vh = np.linalg.svd(T_mat, full_matrices=False)
        k = min(self.chi, len(S))
        U, S, Vh = U[:, :k], S[:k], Vh[:k, :]
        S_sqrt = np.diag(np.sqrt(S))
        C3 = np.einsum('ak,kj->aj', U, S_sqrt).reshape(d, d, k)
        C1 = np.einsum('jk,ka->ja', S_sqrt, Vh).reshape(k, d, d)

        # SVD 2
        T_perm = self.T.transpose(0,3,1,2)
        T_mat = T_perm.reshape(d*d, d*d)
        U, S, Vh = np.linalg.svd(T_mat, full_matrices=False)
        k = min(self.chi, len(S))
        U, S, Vh = U[:, :k], S[:k], Vh[:k, :]
        S_sqrt = np.diag(np.sqrt(S))
        C2 = np.einsum('ak,kj->aj', U, S_sqrt).reshape(d, d, k)
        C0 = np.einsum('jk,ka->ja', S_sqrt, Vh).reshape(k, d, d)
        T_new = np.einsum('Lim,Uji,jkR,mkD->LURD', C0, C1, C2, C3, optimize=True)
        trTT = np.einsum('ijij->', T_new)
        factor = np.abs(trTT)
        self.T = T_new / factor
        self.log_factors.append(np.log(factor))
        self.n_spins.append(2 * self.n_spins[-1])
        self.step += 1

    def output(self):
        path = os.path.join(self.dir, f"{self.step}.npz")
        np.savez(path,
                 T=self.T,
                 log_factors=np.array(self.log_factors),
                 n_spins=np.array(self.n_spins),
                 step=self.step)

    def __next__(self):
        self.update()
        self.output()
        return self

In [3]:
def TRG_gen(temp,chi,N):
    T=TRG(temp,chi)
    for _ in range(N):
        next(T)

# Generate Data

In [4]:
for n in [2,4,8,16,32]:
    for i in np.linspace(0,5,101)[1:]:
        TRG_gen(i,n,20)

In [5]:
for n in [2,4,8,16,32]:
    for i in np.linspace(2,3,101)[1:]:
        TRG_gen(i,n,25)

In [6]:
for n in [2,4,8,16,32]:
    TRG_gen(2.27,n,20)